# Week 10 - Linear Programming in Pyomo

## Learning Objectives
+ Introduction to Pyomo
    + Set 
    + Param
    + Var
    + Objective
    + Constraint
+ Modeling using Pyomo
    + Concrete Model
    + Abstract Model


The contents of this tutorial are based on "Pyomo — Optimization Modeling in Python" by W.E. Hart et. al., pyomo [workshop slides](https://software.sandia.gov/downloads/pub/pyomo/Pyomo-Workshop-Summer-2018.pdf) and pyomo official [documentation](https://pyomo.readthedocs.io/en/stable/).

# Introduction to Pyomo

Pyomo is a Python-based, open-source optimization modeling language with a diverse set of optimization capabilities. It has equation-oriented models, implying that user must provide all model equations explicitly. It is basically a python-based *Algebraic Modeling Language* (AML).

Pyomo can not only specify but solve different types of optimization problems (e.g. linear, quadratic, nonlinear, mixed-integer etc.) by using external solvers.

Solvers are software libraries, mostly written in a lower-level language like c/c++ for faster implementation. Usually, one solver cannot solve all types optimization problem. Using Pyomo, one can access many different solvers depending on the type of problem being solved. Here are few examples of open-source and commercial solvers:

+ Open-Source
    + GLPK: GNU Linear Programming Kit
    + IPOPT: A large scale nonlinear optimizer for continuous systems
+ Proprietary
    + CPLEX: Integer, linear and quadratic programming.
    + Gurobi: Integer, linear and quadratic programming.
    
## Modeling Components in Pyomo
The Pyomo package includes modeling components that are necessary to formulate an optimization problem: variables, objectives, and constraints, as well as other modeling components that are commonly supported by modern AMLs, including index sets and parameters.

+ Var: optimization variables in a model
+ Objective: expressions that are minimized or maximized in a model
+ Constraint: constraint expressions in a model
+ Set: set data that is used to define a model instance
+ Param: parameter data that is used to define a model instance

The basic steps for the modeling process of a problem and getting prescriptive solution can be summarized as below:
1. Convert the task to a mathematical model - i.e. identify its objective and constraints
2. Decide the type of model to use in Pyomo - Abstract or Concrete (usually a personal preference, though some tasks are easier using one than another)
3. Create an instance of a model using Pyomo modeling components - i.e. define the problem using pyomo
4. Pass the instance to solver - solve problem
5. Report and analyze results from the solver

In [31]:
! pip freeze
! pip install pyomo

absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.4
distributed==

In [1]:
import pyomo.environ as pe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# The Warehouse Location Problem

Consider that a company is starting in supply-chain business and wants to build warehouses. They have data about few locations they have visited and evaluated and deemed fit for consideration, but want to find the best warehouse locations, such that all their customers would be servicable. They want to determine feasibility in entire Singapore, without actually investing in going around and analyzing the actual location and it's suitability for the task. Another problem is that as they haven't yet launched, they do not know their exact customers. To reduce their transport and evaluation costs they have approached you as a consultant. How would you approach this problem? 

After thinking about the problem, we realize that the prescriptive part of the problem can be summarized as:

Let $N$ be a set of candidate warehouse locations. We wish to determine the optimal warehouse locations that will minimize the total cost of product delivery to all the customers locations $M$. 

What would be our variables for the model?
+ We would need a discrete variable $y_n$ which is used to indicate if $n$ is selected
+ We would need a variable for the cost of serving customer - $d_{m,n}$, which indicates the cost of delivering product to customer m from warehouse n
+ How do we see all our customers are served? - define a variable $x_{n,m}$ which indicates the fraction of demand for customer $m$ that is served by warehouse $n$.

Let us do the first step - i.e. mathematical model of our problem is to minimise transport and evaluation costs

$\min \limits_{x,y} \sum \limits_{n \in N} \sum \limits_{m \in M} d_{n,m}x_{n,m}$

subject to constraints:

+ All customers are served (by some warehouse): $\sum \limits_{n \in N} x_{n,m} = 1,   \forall m \in M $

+ Each warehouse has the capacity to serve all customers that use that warehouse: $x_{n,m} \leq y_n,   \forall n \in N, m \in M$

+ The number of warehouses built is limited to less than $P$: $\sum \limits_{n \in N} y_n \leq P$

+ The demand for each customer $m$ that is served by each warehouse $n$ is between $0$ and $1$: $ 0\leq x \leq 1$

+ Each warehouse is either open or not open: $ y \in \{0,1\}$




Let us define the costs $d_{n,m}$ as follows ( the columns give the customer locations while rows give candidate warehouse locations)

||Bukit Batok| Downtown| Changi| Clementi|
|--|--|--|--|--|
|Paya Lebar| 1956| 1606 | 1410 | 330|
|Woodlands| 1096| 1792| 531| 567|
|Pasir Ris| 485| 2322| 324| 1236|

# Loading Data

This data is available in delivery_costs.csv. Let us load the data.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IT5006/Week 10/delivery_costs.csv', index_col=0)

In [4]:
df.head()

,Bukit Batok,Downtown,Changi,Clementi
Paya Lebar,1956,1606,1410,330
Woodlands,1096,1792,531,567
Pasir Ris,485,2322,324,1236


The variables $x$ and $y$ are determined by the optimizer, and all other quantities are
known inputs or parameters in the problem. This problem is a particular description of the [p-median problem](http://www.math.nsc.ru/AP/benchmarks/P-median/p-med_eng.html), and it has the interesting property that the $x$ variables will converge to $\{0, 1\}$ even though they are not specified as binary variables.

# Concrete Models vs Abstract Models in Pyomo

Let us first mathematically understand the difference between Concrete and Abstract modeling in Pyomo.

$ \min \sum_{j=1}^n c_j x_j $

such that

$ \sum_{j=1}^n a_{ij} x_j \geq b_i,    \forall i = 1 \ldots m $

$x_j \geq 0,      \forall j = 1 \ldots n$

We call this an abstract or symbolic mathematical model since it relies on *unspecified parameter values*. Data values can be used to specify a model instance. The ```AbstractModel``` class provides a context for defining and initializing abstract optimization models in Pyomo when the data values will be supplied at the time a solution is to be obtained.

In many contexts, a mathematical model can and should be *directly defined with the data values supplied at the time of the model definition*. We call these concrete mathematical models. For example, consider the following:

$\min    2 x_1 + 3 x_2$

such that

$ 3 x_1 + 4 x_2 \geq 1$

$ x_1, x_2 \geq 0$

The ```ConcreteModel``` class is used to define concrete optimization models in Pyomo.

<div class="alert alert-block alert-info"> 
    <b>NOTE</b> 
    Python programmers will probably prefer to write concrete models, while users of some other algebraic modeling languages may tend to prefer to write abstract models. The choice is largely a matter of taste; some applications may be a little more straightforward using one or the other.
</div>

# Concrete Model

We will first define the problem using ```ConcreteModel```, and then move to model using ```AbstractModel```. We can specify a name for the model.

In [5]:
model = pe.ConcreteModel(name="(WL)")

## Defining Pyomo Components: Variables

Optimization problems require, at least, one variable and an objective function. Most problems also include constraints. For each optimization variable, we create an instance of the ```Var``` class and add that instance as an attribute to the ```model``` object. The code ```model.x=Var()``` creates an instance of the Pyomo class ```Var ``` and makes it accessible by ```model.x```.

In ```Var```,  keyword arguments can be used to define properties of the variable. For example, ```bounds``` is used to set lower and upper bounds, ```initialize``` is used to set initial values, and ```within``` is used
to set the domain.

If the domain is not specified, the default is the ```Reals``` virtual set. Other virtual sets supported by Pyomo are: ```PositiveReals```, ```Integers```, ```Boolean```, etc. The complete list can be found [here](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html#predefined-virtual-sets).



Notice in our problem, the variable and constraints are indexed through the data provided. We can create separate $x$ variables for each pair of warehouse and customers. However, a more elegant option is to use construction rules. So, we first need to create the data in Python, which can be used to provide Pyomo with list of valid indices for ```x``` and ```y``` variables.

In [6]:
N = list(df.index.map(str)) # warehouse locations
M = list(df.columns.map(str)) # customer locations
d = {(r, c):df.at[r,c] for r in N for c in M} # the cost at each warehouse location and customer location

For our problem, let us say that the company wants to only build two warehouses in the beginning.

In [7]:
P = 2

In [8]:
model.x = pe.Var(N, M, bounds=(0,1)) # the warehouse locations and customer locations are x variables
model.y = pe.Var(N, within=pe.Binary) # the warehouse (built) is the y variable

We refer to ```N``` and ```M``` as index sets for the indexed variables ```model.x``` and ```model.y```.
Specifically, the variable y is indexed over N, and the variable x is a two-dimensional array that is indexed over both N and M. With this declaration, an element of x can be accessed by ```model.x[i,j]``` where i and j are elements of the sets N and M, respectively.

## Defining Pyomo Components - Constraints and Objectives

### Defining Constraint
Nearly all of Pyomo’s modeling components can be indexed, and the construction of many indexed constraints is performed with construction rules. An alternative strategy usable only in ```ConcreteModel``` class is the use of ```expr```. The ```Objective``` or ```Constraint``` can be passed an expression rather than an indexed rule. Let us illustrate this for the simple exaple used earlier to explain a Concrete Model in Pyomo.

```
## for the simple example 

model = pe.ConcreteModel()

model.x = pe.Var([1,2], domain=pe.NonNegativeReals)

model.OBJ = pe.Objective(expr = 2*model.x[1] + 3*model.x[2])

model.Constraint1 = pe.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)
```

You might want to use this same style for the problem at hand too, and you are free to do so, however, a ```rule```-based approach is more scalable for our task at hand.

Consider our constraint, 

$\sum \limits_{n \in N} x_{n,m} = 1,   \forall m \in M $

This mathematical notation indicates that there is a single constraint for each $m$ in the set $M$. The ```Constraint``` component can be declared as an indexed constraint over the elements in this set. However, we need a mechanism to provide Pyomo with the explicit expressions for each element in $M$. Pyomo allows model components to be initialized with user-defined functions, which we call ```rules```.


In [9]:
def one_per_cust_rule(model, m):
    return sum(model.x[n,m] for n in N) == 1
model.one_per_cust = pe.Constraint(M, rule=one_per_cust_rule)

The last line in this example declares the constraint by creating a ```Constraint``` component that is indexed over the set ```M```. The ```rule``` keyword argument indicates that the function ```one_per_cust_rule``` is used to construct each constraint. 

We can create another constraint for the warehouse capacity, and call it ```warehouse_active_rule```. 

Each warehouse has the capacity to serve all customers that use that warehouse: $x_{n,m} \leq y_n,   \forall n \in N, m \in M$



In [10]:
def warehouse_active_rule(model, n, m):
   return model.x[n,m] <=model.y[n]
model.warehouse_active = pe.Constraint(N, M, rule=warehouse_active_rule)

Lastly we create a rule for the total number of warehouses. 

The number of warehouses built is limited to less than $P$: $\sum \limits_{n \in N} y_n \leq P$




In [11]:
def num_warehouses_rule(model):
    return sum(model.y[n] for n in N) <= P
model.num_warehouses = pe.Constraint(rule=num_warehouses_rule)

Pyomo expects a construction rule to return an expression for every index value. If no constraint is needed for a particular combination of indices, then the value ```Constraint.Skip``` can be returned instead.

## Defining Objective

In [12]:
def obj_rule(model):
    return sum(d[n,m]*model.x[n,m] for n in N for m in M)
model.obj = pe.Objective(rule=obj_rule)

## Solving the Model

It is good to note that ```pyomo``` command can be also directly used in command line to solve a model defined in python script by:
```
pyomo solve --solver-glpk script.py data.csv
```
Note here, that the script would need to take the system-argument to use the data file.

However, let us continue to use the python API so that the entire pipeline of our analysis can be kept in python for convenience. We use ```SolverFactory``` to mention the solver to be used for pyomo.

Pyomo supports a wide variety of solvers. Pyomo has specialized interfaces to some solvers (for example, BARON, CBC, CPLEX, and Gurobi). It also has generic interfaces that support calling any solver that can read AMPL “.nl” and write “.sol” files and the ability to generate GAMS-format models and retrieve the results. We use the GLPK (GNU Linear Programming Kit) solver.

In [13]:
model.pprint()

8 Set Declarations
    one_per_cust_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Bukit Batok', 'Downtown', 'Changi', 'Clementi'}
    warehouse_active_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                                            : Size : Members
        None :     2 : warehouse_active_index_0*warehouse_active_index_1 :   12 : {('Paya Lebar', 'Bukit Batok'), ('Paya Lebar', 'Downtown'), ('Paya Lebar', 'Changi'), ('Paya Lebar', 'Clementi'), ('Woodlands', 'Bukit Batok'), ('Woodlands', 'Downtown'), ('Woodlands', 'Changi'), ('Woodlands', 'Clementi'), ('Pasir Ris', 'Bukit Batok'), ('Pasir Ris', 'Downtown'), ('Pasir Ris', 'Changi'), ('Pasir Ris', 'Clementi')}
    warehouse_active_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Paya Lebar', 'Woodlands', 'Pasir Ris'}
    warehouse_active

In [14]:
!apt-get install -y -qq glpk-utils

In [15]:
solver = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')
solver.solve(model)
model.display()

Model '(WL)'

  Variables:
    x : Size=12, Index=x_index
        Key                           : Lower : Value : Upper : Fixed : Stale : Domain
         ('Pasir Ris', 'Bukit Batok') :     0 :   1.0 :     1 : False : False :  Reals
              ('Pasir Ris', 'Changi') :     0 :   1.0 :     1 : False : False :  Reals
            ('Pasir Ris', 'Clementi') :     0 :   0.0 :     1 : False : False :  Reals
            ('Pasir Ris', 'Downtown') :     0 :   0.0 :     1 : False : False :  Reals
        ('Paya Lebar', 'Bukit Batok') :     0 :   0.0 :     1 : False : False :  Reals
             ('Paya Lebar', 'Changi') :     0 :   0.0 :     1 : False : False :  Reals
           ('Paya Lebar', 'Clementi') :     0 :   1.0 :     1 : False : False :  Reals
           ('Paya Lebar', 'Downtown') :     0 :   1.0 :     1 : False : False :  Reals
         ('Woodlands', 'Bukit Batok') :     0 :   0.0 :     1 : False : False :  Reals
              ('Woodlands', 'Changi') :     0 :   0.0 :     1 : False : 

## Getting Results in Human Readable Form

Let us convert the solved model, to text-based prescriptions. After all, this is what is requested from us as consultants!

In [16]:
for wl in N:
    if pe.value(model.y[str(wl)]) > 0.5:
        customers = [str(cl) for cl in M if pe.value(model.x[wl, cl] > 0.5)]
        print(str(wl)+' serves customers: '+str(customers))
    else:
        print(str(wl)+": do not build")

Paya Lebar serves customers: ['Downtown', 'Clementi']
Woodlands: do not build
Pasir Ris serves customers: ['Bukit Batok', 'Changi']


# Abstract Model

Now, let us solve this same problem using ```AbstractModel``` class in pyomo. Similar to before, we can define name for the model. Abstract models require declarations for sets and parameters that are used in the
model.

In [17]:
model = pe.AbstractModel(name="(WL)")

## Defining Pyomo Components: Set and Param

## Defining Set

A Pyomo ```Set``` component is used to declare valid indices for any component that is indexed. For example, in the context of our warehouse location problem, we have two sets: N stores the valid warehouse locations and M stores the customer locations. These ```Set``` objects can then be used to define indexed variables or constraints.

In [18]:
model.N = pe.Set()
model.M = pe.Set()

We pass ```Set``` objects into the ```Var``` constructor, rather than the Python lists used for the concrete model in the previously. A ```Set``` component can be initialized with the ```initialize``` keyword argument, using a Python set, list, or tuple. The ```Set``` component can also be initialized with external data sources. Pyomo Set objects can also be indexed by other sets. You can read other details in the [documentation](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html).

In [19]:
model.x = pe.Var(model.N, model.M, bounds=(0,1))
model.y = pe.Var(model.N, within=pe.Binary)

## Defining Param

A Pyomo ```Param``` component is used to define data values for our problem. In the context of the warehouse location problem, two pieces of data need to be specified: $P$ and $d_{n,m}$. These parameters can be scalar or indexed.


By default, parameters are immutable, meaning that once their values are set, the values cannot be changed. This default behavior allows for increased efficiency within Pyomo when handling expressions. However, you can define a parameter whose values are mutable with the ```mutable=true``` keyword argument. This can be useful if you have a model that you want to solve multiple times with different values of some of the parameters.


In [20]:
model.d = pe.Param(model.N,model.M) # can be indexed
model.P = pe.Param() # scalar

## Defining Objective and Constraints

The coding for the objective and constraints would remain same for our case, as we had used ```rule``` for defining both in case of ```ConcreteModel``` too. However, as this is an abstract model, the objective rule ```obj_rule``` would not be called just yet. On defining those, Pyomo will know what rule to call to construct the objective component, but it has not called the constructor because this is an abstract model. Constraint rules and constraint components are also declared in a similar manner.


In [21]:
def obj_rule(model):
    return sum(d[n,m]*model.x[n,m] for n in N for m in M)
model.obj = pe.Objective(rule=obj_rule)

def one_per_cust_rule(model, m):
    return sum(model.x[n,m] for n in N) == 1
model.one_per_cust = pe.Constraint(M, rule=one_per_cust_rule)

def warehouse_active_rule(model, n, m):
    return model.x[n,m] <= model.y[n]
model.warehouse_active = pe.Constraint(N, M, rule=warehouse_active_rule)

def num_warehouses_rule(model):
    return sum(model.y[n] for n in N) <= P
model.num_warehouses = pe.Constraint(rule=num_warehouses_rule)

## Managing Data in Abstract Models

Uptill now, our implementation has declared the model, but it does *do* anything. This is because it does not define the model data or solver instance. The action of applying a data file to this abstract model can be scripted explicitly in Python code, or it can be done using the ```pyomo``` command.

Let us first define the model data using standard Python object: ```dict```. There are other methods of loading data too, and you can read about those in the [documentation for data loading](https://pyomo.readthedocs.io/en/stable/working_abstractmodels/data/index.html).

Data can be passed to the model ```create_instance()``` method through a series of nested native Python dictionaries. The structure begins with a dictionary of namespaces, with the only required entry being the ```None``` namespace. Each namespace contains a dictionary that maps component names to dictionaries of component values. For scalar components, the required data dictionary maps the implicit index ```None``` to the desired value

In [22]:
data = {None: 
    {'N': {None: list(df.index.map(str))},
    'M': {None: list(df.columns.map(str))},
    'd': {(r, c):df.at[r,c] for r in N for c in M},
    'P': {None: 2}}
}

In [23]:
model.pprint()

8 Set Declarations
    M : Size=0, Index=None, Ordered=Insertion
        Not constructed
    N : Size=0, Index=None, Ordered=Insertion
        Not constructed
    d_index : Size=0, Index=None, Ordered=True
        Not constructed
    one_per_cust_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Bukit Batok', 'Downtown', 'Changi', 'Clementi'}
    warehouse_active_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                                            : Size : Members
        None :     2 : warehouse_active_index_0*warehouse_active_index_1 :   12 : {('Paya Lebar', 'Bukit Batok'), ('Paya Lebar', 'Downtown'), ('Paya Lebar', 'Changi'), ('Paya Lebar', 'Clementi'), ('Woodlands', 'Bukit Batok'), ('Woodlands', 'Downtown'), ('Woodlands', 'Changi'), ('Woodlands', 'Clementi'), ('Pasir Ris', 'Bukit Batok'), ('Pasir Ris', 'Downtown'), ('Pasir Ris', 'Changi'), ('Pasir Ris', 'Clementi')}
    w

In [24]:
instance = model.create_instance(data=data)

We can see the parameters and the values assigned to them.

In [25]:
for parmobject in instance.component_objects(pe.Param, active=True):
    print ("Parameter ", parmobject.name)
    for index, value in parmobject.iteritems(): # like a dictioanry
        print ("   ",index, pe.value(value))

Parameter  d
    (deprecated in 6.0) (called from /usr/local/lib/python3.7/dist-
    packages/IPython/core/interactiveshell.py:2882)
    ('Paya Lebar', 'Bukit Batok') 1956
    ('Paya Lebar', 'Downtown') 1606
    ('Paya Lebar', 'Changi') 1410
    ('Paya Lebar', 'Clementi') 330
    ('Woodlands', 'Bukit Batok') 1096
    ('Woodlands', 'Downtown') 1792
    ('Woodlands', 'Changi') 531
    ('Woodlands', 'Clementi') 567
    ('Pasir Ris', 'Bukit Batok') 485
    ('Pasir Ris', 'Downtown') 2322
    ('Pasir Ris', 'Changi') 324
    ('Pasir Ris', 'Clementi') 1236
Parameter  P
    None 2


## Solving the Pyomo Model

Just as in the case of the ```ConcreteModel```, we use ```SolverFactory``` and GLPK to solve our problem. We expect both our outputs to be the same.

In [26]:
solver = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')
solver.solve(instance)
instance.display()

Model '(WL)'

  Variables:
    x : Size=12, Index=x_index
        Key                           : Lower : Value : Upper : Fixed : Stale : Domain
         ('Pasir Ris', 'Bukit Batok') :     0 :   1.0 :     1 : False : False :  Reals
              ('Pasir Ris', 'Changi') :     0 :   1.0 :     1 : False : False :  Reals
            ('Pasir Ris', 'Clementi') :     0 :   0.0 :     1 : False : False :  Reals
            ('Pasir Ris', 'Downtown') :     0 :   0.0 :     1 : False : False :  Reals
        ('Paya Lebar', 'Bukit Batok') :     0 :   0.0 :     1 : False : False :  Reals
             ('Paya Lebar', 'Changi') :     0 :   0.0 :     1 : False : False :  Reals
           ('Paya Lebar', 'Clementi') :     0 :   1.0 :     1 : False : False :  Reals
           ('Paya Lebar', 'Downtown') :     0 :   1.0 :     1 : False : False :  Reals
         ('Woodlands', 'Bukit Batok') :     0 :   0.0 :     1 : False : False :  Reals
              ('Woodlands', 'Changi') :     0 :   0.0 :     1 : False : 

In [27]:
instance.pprint()

8 Set Declarations
    M : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Bukit Batok', 'Downtown', 'Changi', 'Clementi'}
    N : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Paya Lebar', 'Woodlands', 'Pasir Ris'}
    d_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    N*M :   12 : {('Paya Lebar', 'Bukit Batok'), ('Paya Lebar', 'Downtown'), ('Paya Lebar', 'Changi'), ('Paya Lebar', 'Clementi'), ('Woodlands', 'Bukit Batok'), ('Woodlands', 'Downtown'), ('Woodlands', 'Changi'), ('Woodlands', 'Clementi'), ('Pasir Ris', 'Bukit Batok'), ('Pasir Ris', 'Downtown'), ('Pasir Ris', 'Changi'), ('Pasir Ris', 'Clementi')}
    one_per_cust_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Bukit Bato

In [28]:
for wl in N:
    if pe.value(instance.y[str(wl)]) > 0.5:
        customers = [str(cl) for cl in M if pe.value(instance.x[wl, cl] > 0.5)]
        print(str(wl)+' serves customers: '+str(customers))
    else:
        print(str(wl)+": do not build")

Paya Lebar serves customers: ['Downtown', 'Clementi']
Woodlands: do not build
Pasir Ris serves customers: ['Bukit Batok', 'Changi']


The ```display``` function displays values in the block, while the ```pprint``` is used for printing component information. So, the ```display``` function will not work until the component has been constructed and so, ```pprint``` is useful for debugging.

In [ ]:
ValueError Failed to set executable for solver glpk.
